In [1]:
import sys 
import os
import  numpy as np

outputFile="outputPS4.txt"
inputFile="inputPS4.txt"
commandFile="promptsPS4.txt"

class PS4:
    vertices = []
    edges = []
    TrainEdges = {}
    TrainEdges_1 = {}
    CityTrains = {}
    
    
    #read the input file 
    def __init__(self):
         self.readCityTrainfile(inputfile=inputFile)
        
    #add each station as vertex        
    def addVertex(self, v):
        if not self.isVertex(v):
            self.vertices.append(v)
    
    # verify whether a vertex (station) already added/present
    def isVertex(self, v):
        if v in self.vertices:
            return True
     
    # add route between 2 stations with train number
    def addEdges(self, u, v):
        self.edges.append((u,v))        
    
    # findout whether 2 stations are connected Or not
    def areAdjacent(self, u, v):
        if ((u,v) in self.edges or (v,u) in self.edges):
            return True
    
    # findout whether there is a train between 2 stations
    def findTrainBetweenCities(self, u, v):
        trains = []
        if self.areAdjacent(u, v):
            for train, cities in self.TrainEdges_1.items():
                if((u, v) in cities):
                    trains.append(train)
            return trains
        else:            
            return None
    
    # findout whether there is a train between 2 stations
    def findAdjacentVertices(self, u):
        adjVertices = []
        for i in self.vertices:
            if self.areAdjacent(u, i):
                adjVertices.append(i)                
        return adjVertices
        
    def addTrainEdges(self, train, e):
        self.TrainEdges[train] = e
    
    def addTrainEdges_1(self, train, u, v):
        self.addEdges(u, v)
        if train in self.TrainEdges_1:
            self.TrainEdges_1[train].append((u,v))
        else:
            self.TrainEdges_1[train] = [(u,v)]
    
    # associate trains with cities
    def addCityTrains(self, v, trains):
        if v in self.CityTrains:
            self.CityTrains[v].append(trains)
        else:
            self.CityTrains[v] = [trains]
    
    def findPath(self, start, end, path=[]):
        path = path + [start]
        if start == end:
            print(path)
            return path
        if not self.isVertex(start):
            return None
        for node in self.findAdjacentVertices(start):
            if node not in path:
                newpath = self.findPath(node, end, path)
                if newpath: return newpath
        return None
    
     # find  all the paths between given 2 stations if exists
    def findAllPaths(self, start, end, path=[]):
        path = path + [start]
        if start == end:
            return [path]
        if not self.isVertex(start):
            return []
        paths = []
        for node in self.findAdjacentVertices(start):
            if node not in path:
                #trains = findTrainBetweenCities(start, node)
                newpaths = self.findAllPaths(node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths
    
    # find the shortest path between given 2 stations if exists
    def findShortestPath(self, start, end, path=[]):
        path = path + [start]
        if start == end:
            print(path)
            return path
        if not self.isVertex(start):
            return None
        shortest = None
        for node in self.findAdjacentVertices(start):
            if node not in path:
                newpath = self.findShortestPath(node, end, path)
                if newpath:
                    if not shortest or len(newpath) < len(shortest):
                        shortest = newpath
        return shortest
        
     # reading the input file , actual biggining of the execution   
    def  readCityTrainfile(self, inputfile):
        f = open(inputfile,"r")
        #contents =f.read()
        Lines = f.readlines()
        for i in Lines:
            i = i.strip()
            fields = i.split ('/')
            train_edges = []
            counter = 0
            prev_fld = "NIL"
            for fld in fields:
                fld = fld.strip()
                if counter == 0:
                    train = fld
                    #print(i)  
                else:
                    self.addVertex(fld)    
                    self.addCityTrains(fld, train)
                    train_edges.append(fld)
                    self.addTrainEdges_1(train, prev_fld, fld)
                    prev_fld = fld
                    
                self.addTrainEdges(train, train_edges)                
                #self.TrainEdges.add(train, e)
                counter = counter + 1
          
        f.close()

    # Show the content of the file
    def showAll(self):
        f = open(outputFile, "w")
        f.write("--------Function showAll --------\n")
        f.write("Total no. of freight trains: "+str(len(self.TrainEdges))+"\n")
        f.write("Total no. of cities: "+str(len(self.vertices))+"\n")
        f.write("List of Freight trains:\n")
        for i in self.TrainEdges.keys():
            f.write(i+"\n")
        f.write("List of cities:\n")
        for i in self.vertices:
            f.write(i+"\n") 
        f.write("---------------------------------------\n")
        f.close()
    
    # find and display the transport hub , which  city has maximun number of trains are passing by    
    def displayTransportHub(self):
        print("\n")
        maxCount = 0
        maxCity = "NONE"
        for i in self.vertices:
            count= 0
            for j in self.vertices:
                #print(i,j)
                if self.areAdjacent(i,j):
                    count = count + 1
            #print(i," : ", count)
            if count > maxCount:
                maxCount = count
                maxCity = i
        #print("TransportHub : ",maxCity, " - ", maxCount)
        #print(self.CityTrains[maxCity])
        f = open(outputFile, "a")
        f.write("\n--------Function displayTransportHub --------\n")
        f.write("Main transport hub: "+maxCity+"\n")
        f.write("Number of trains visited: "+str(maxCount)+"\n")
        f.write("List of Freight trains:"+"\n")
        trainsAtHub = np.array(self.CityTrains[maxCity]) 
        unique_trainsAtHub = np.unique(trainsAtHub)
        for trn in unique_trainsAtHub:
               f.write(trn+"\n")
        f.write("---------------------------------------\n")
        f.close
    
    # Display the cities that are connected  by a train
    def displayConnectedCities(self, train):
        #print(self.TrainEdges[train])
        f = open(outputFile, "a")
        f.write("\n--------Function displayConnectedCities --------\n")
        f.write("Freight train number: "+train+"\n")
        try:
            trainEdges = self.TrainEdges[train]
        except KeyError:
            f.write('Train No : {} doesnot exist in the list of available trains , hence invalid train number\n'.format(train))
            f.close
            return
        f.write("Number of cities connected: "+str(len(trainEdges))+"\n")
        f.write("List of cities connected directly by "+train+":\n")
        for city in trainEdges:
             f.write(city+"\n")
        f.write("---------------------------------------\n")
        f.close
    
    # find and display if there is a drirect train between given 2 cities
    def  displayDirectTrain(self, city_a, city_b):
        f = open(outputFile, "a")
        f.write("\n--------Function displayDirectTrain --------\n")
        f.write("City A: "+city_a+"\n")
        f.write("City B: "+city_b+"\n")
        try:
            if not self.isVertex(city_a):
                f.write('{} is not part of the list of cities covered by frieght path\n'.format(city_a))
                f.write("---------------------------------------\n")
                raise
            if not self.isVertex(city_b):
                f.write('{} is not part of the list of cities covered by frieght path\n'.format(city_b))
                f.write("---------------------------------------\n")
                raise
        except:
            f.close
            return        
        trains = []  
        if self.areAdjacent(city_a, city_b):
            trains = self.findTrainBetweenCities(city_a, city_b)
            f.write("Package can be sent directly: Yes - Available trains are -"+','.join(str(e) for e in trains)+"\n")
        else:
            f.write("Package can be sent directly: No, there is no direct train between these two cities."+"\n")

        f.write("---------------------------------------\n")
        f.close
        return
        
    
    def findServiceAvailable(self, u, v):
        f = open(outputFile, "a")
        f.write("\n--------Function displayDirectTrain --------\n")
        f.write("City A: "+u+"\n")
        f.write("City B: "+v+"\n")
        try:
            if not self.isVertex(u):
                f.write('{} is not part of the list of cities covered by frieght path\n'.format(u))
                f.write("---------------------------------------\n")
                raise
            if not self.isVertex(v):
                f.write('{} is not a valid city. Please pass a proper city name in order to get the result\n'.format(v))
                f.write("---------------------------------------\n")
                raise
        except:
            f.close
            return
            
        paths = self.findAllPaths(u, v) 
        if len(paths) > 0:
            str_path = ""
            if len(paths) == 1:
                f.write("\nCan the package be sent: Yes, direct train is available\n")
            else:
                f.write("\nCan the package be sent: Yes. There are {} path/(s) available. Below are the details.\n".format(len(paths)))
            for path in paths:
                counter = 0
                for city in path:
                    if counter == 0:                    
                        if len(paths) == 1:
                            str_path = str_path + city
                        else:
                            str_path = str_path + "\n"+ city
                        prev_city = city
                        counter = counter + 1
                    else:
                        trains = self.findTrainBetweenCities(prev_city, city)
                        str_path = str_path + " > " + ', '.join(str(e) for e in trains) + " > " + city
                        prev_city = city
            f.write(str_path)                
        else:
            f.write("\n\nCan the package be sent: No, No route available between {} and {}.\n\n".format(u,v))
        f.close        
    

# initializing the class
graph_1 = PS4()

graph_1.readCityTrainfile(inputFile)
graph_1.showAll()

# read command file
cmdfile = open(commandFile, "r")

for instruction in cmdfile:
    fields = instruction.split(':')
    
    if  fields[0] == 'searchTransportHub' :
        graph_1.displayTransportHub()
    elif fields[0] == 'searchTrain' :
        graph_1.displayConnectedCities(fields[1].strip())
    elif fields[0] == 'searchCities' :
        graph_1.displayDirectTrain(fields[1].strip(),fields[2].strip())
    elif fields[0] == 'ServiceAvailability' :
        graph_1.findServiceAvailable(fields[1].strip(),fields[2].strip())
    else:
            f = open(outputFile, "a")
            f.write("\n-------- Valid command list --------\n")
            f.write("searchTransportHub:\n")
            f.write("searchTrain:< train number>\n")
            f.write("searchCities:<city a>:<city b>\n")
            f.write("ServiceAvailability:<city a>:<city b>\n")
           
            f.close()

cmdfile.close()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Mallik\\Desktop\\Python_Code\\inputPS4.txt'